In [30]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split,Dataset,Subset
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import optuna
import joblib

In [31]:
cuda = True if torch.cuda.is_available() else False

In [32]:
train=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/train.csv")
test=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/test.csv")

In [33]:
class FADataset(Dataset):
    def __init__(self, data,train=True):
        super().__init__()
        if(train):
            self.x=data.drop(['id','y'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            self.y=torch.tensor(data['y'].values,dtype=torch.float32)
        else:
            self.x=data.drop(['id'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data=self.x[idx]
        target=self.y[idx]
        return data,target,idx


In [34]:
dataset=FADataset(train,train=True)
trainset, validset = random_split(dataset, [0.9, 0.1])
testset=FADataset(test,train=False)
test_dl=DataLoader(testset,shuffle=False, batch_size=80)

In [102]:
class MLP(nn.Module):
    def __init__(self,params):
        super().__init__()
        self.model=nn.Sequential(nn.Linear(2,params['num_hidden1']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden1'],params['num_hidden2']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden2'],params['num_hidden3']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden3'],params['num_hidden4']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden4'],params['num_hidden5']),
                                nn.ReLU(True),  
                                nn.Linear(params['num_hidden5'],params['num_hidden6']),
                                nn.Linear(params['num_hidden6'],1)
                                )
    def forward(self, x):
        return self.model(x)


In [103]:
def train_and_evaluate(params,model,datadl):

    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    
    criterion = nn.MSELoss()

    losses=[]
    num_epochs=50
    model.train()
    for epoch in range(num_epochs):
        for batch,(data, target,idx) in enumerate(datadl):
            outputs = model(data)
            loss = criterion(outputs.reshape(-1), target.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    model.eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(datadl):
        outputs = model(data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    # print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)
    
    return model,loss


In [104]:

def objective(trial):
        
        params = {

            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
            'batch_size': trial.suggest_int("batch_size", 64, 320,step=16),
            'num_hidden1': trial.suggest_int("num_hidden1", 50, 500),
            'num_hidden2': trial.suggest_int("num_hidden2", 50, 600),
            'num_hidden3': trial.suggest_int("num_hidden3", 50, 600),
            'num_hidden4': trial.suggest_int("num_hidden4", 50, 600),
            'num_hidden5': trial.suggest_int("num_hidden5", 50, 600),
            'num_hidden6': trial.suggest_int("num_hidden6", 50, 600)
            }
        

        train_dl= DataLoader(trainset, shuffle=True, batch_size=params['batch_size'])
        valid_dl= DataLoader(validset, shuffle=True, batch_size=params['batch_size'])


        model= MLP(params)
        best_model,loss=train_and_evaluate(params,model,train_dl)
      
        torch.save(best_model.state_dict(), f"model_trial_{trial.number}.pth")
    
        return loss


In [105]:
train_dl= DataLoader(trainset, shuffle=True, batch_size=study_alldata.best_params['batch_size'])
valid_dl= DataLoader(validset, shuffle=True, batch_size=study_alldata.best_params['batch_size'])
criterion = nn.MSELoss()

In [109]:
model=[]
for i in range(5):
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study.optimize(objective, n_trials=30)
    print(f"model_{i} loss=",study.best_value)
    model.append(MLP(study.best_params))
    model[i].load_state_dict(torch.load(f"C:/Users/MJ/Desktop/FunctionApproximation/model_trial_{study.best_trial.number}.pth"))
    torch.save(model[i],f'model_{i}.pth')

model_0 loss= 0.023185842164925165
model_1 loss= 0.02315875579627742
model_2 loss= 0.02304524197284363
model_3 loss= 0.022980503556607045
model_4 loss= 0.024481509402394293


In [110]:
for i in range(5):
    model[i].eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(valid_dl):
        outputs = model[i](data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)

loss: 0.021460 
loss: 0.022042 
loss: 0.022466 
loss: 0.021388 
loss: 0.023067 


In [127]:
def objective_en(trial):
    a=trial.suggest_float('a',0, 100)
    b=trial.suggest_float('b',0, 100)
    c=trial.suggest_float('c',0, 100)
    d=trial.suggest_float('d',0, 100)
    e=trial.suggest_float('e',0, 100)
    
    losses=[]
    
    for batch,(data, target,idx) in enumerate(train_dl):
        outputs = (a*model[0](data)+b*model[1](data)+c*model[2](data)+d*model[3](data)+e*model[4](data))/(a+b+c+d+e)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    # print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)

    return loss

study_en = optuna.create_study(direction='minimize')
study_en.optimize(objective_en, n_trials=100)


In [129]:
study_en.best_value

0.022244182217503544

In [130]:
a=study_en.best_params['a']
b=study_en.best_params['b']
c=study_en.best_params['c']
d=study_en.best_params['d']
e=study_en.best_params['e']

output=[]

for i in range(len(testset)):
    output.append(((a*model[0](testset.x[i])+b*model[1](testset.x[i])+c*model[2](testset.x[i])+d*model[3](testset.x[i])+e*model[4](testset.x[i]))/(a+b+c+d+e)).item())

sample_submission=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/sample_submission.csv")
sample_submission['y']=output
sample_submission.to_csv("C:/Users/MJ/Desktop/sample_submission.csv",index=False)

In [ ]:
# model_0.eval()

# for batch,(data, target,idx) in enumerate(valid_dl):
#     outputs = model_0(data)
#     loss = criterion(outputs.reshape(-1), target.reshape(-1))
# print(f"loss: {loss.item():>7f} ")